### Código

1. Selecionar sinais e coletar seus pontos ao longo do tempo
    - $\sin(x)$
    - $\cos(x)$
    - $c_2 \sin(x) + c_1 \cos(x)$
    - Onda quadrada
    - Onda triangular
    - Adicionar ruído nas "funções-ingredientes": $c_2 (\sin(x) + rand) + c_1 \cos(x) + c_0$
    - Adicionar ruído nas funções "finais": $c_2 \sin(x) + c_1 \cos(x) + c_0 + rand$
2. Criar uma matriz com esses pontos (tempo, amplitude)
3. Aplicar SVD
    - Redução de posto
        - Visualizar diferença da matriz aproximada com a original
        - Mostrar erro em valor numérico
    - Visualização dos left singular vectors (Matriz U)
4. Redução de dimensão (analisar grupos)
5. Aplicações
    - Vozes ou instrumentos

obs: Talvez seja uma boa normalizar os sinais antes de aplicar o SVD

### Apresentação

1. O que são sinais e sua relevância
2. Como analisar esses sinais (Fourier X SVD)
3. Mostrar código 
4. Apresentar resultados (gráficos) do SVD em sinais simples (Focar o que o SVD faz com os sinais)
5. Aplicações (Classificador)

Possíveis resultados
1. Remoção de ruído
2. Decomposição dos sinais
3. Classificação dos tipos de sinais


## Sinais

In [ ]:
using Plots
using LinearAlgebra
using Random
using WAV
using Random, Distributions

In [ ]:
function signal_vec(func, sample_points, noise_level)
    return [func(x) + noise_level * rand(Normal(0, 2)) for x in sample_points]
end

In [ ]:
function signal_matrix(func, sample_points, noise_level, num_signals)
    return hcat([signal_vec(func, sample_points, i*noise_level) for i in 1:num_signals]...)
end

### $\sin(x)$

In [ ]:
# Exemplo de sinal sin com ruído

sample_points = 0:0.1:10 # pontos de amostragem
num_signals = 10 # número de sinais
noise_level = 0.01 # nível de ruído

sin_signal = signal_matrix(sin, sample_points, noise_level, num_signals)
sin_signal_raw = signal_matrix(sin, sample_points, 0, num_signals)

plot(sample_points, sin_signal, label="Sinais", xlabel="Tempo", ylabel="Amplitude", title="Sinais senoidais com ruído")

### $cos(x/2)$

In [ ]:
# Exemplo de sinal sin com ruído

sample_points = 0:0.1:10 # pontos de amostragem
num_signals = 10 # número de sinais
noise_level = 0.01 # nível de ruído

cos_signal = signal_matrix(x -> cos(x/2), sample_points, noise_level, num_signals)
cos_signal_raw = signal_matrix(x -> cos(x/2), sample_points, 0, num_signals)

plot(sample_points, cos_signal, label="Sinais", xlabel="Tempo", ylabel="Amplitude", title="Sinais cos(x/2) com ruído")

### $rand(x)$

In [ ]:
# Exemplo de sinal aleatório com ruído

sample_points = 0:0.1:10 # pontos de amostragem
num_signals = 10 # número de sinais
noise_level = 0.01 # nível de ruído

rand_signal = signal_matrix(x -> rand(), sample_points, noise_level, num_signals)
rand_signal_raw = signal_matrix(x -> rand(), sample_points, 0, num_signals)

plot(sample_points, rand_signal, label="Sinais", xlabel="Tempo", ylabel="Amplitude", title="Sinais aleatórios com ruído")

### Sinais com impulso

In [ ]:
# Exemplo de sinal impulso com ruído

sample_points = 0:0.1:10 # pontos de amostragem
num_signals = 10 # número de sinais
noise_level = 0.01 # nível de ruído

impulse_signal = signal_matrix(x -> x == 5 ? 1 : 0, sample_points, noise_level, num_signals)
impulse_signal_raw = signal_matrix(x -> x == 5 ? 1 : 0, sample_points, 0, num_signals)

plot(sample_points, impulse_signal, label="Sinais", xlabel="Tempo", ylabel="Amplitude", title="Sinais impulso com ruído")

### $\sin(x) + \cos(3x)$

In [ ]:
# Sinal Sin + Cos 

sample_points = 0:0.1:10 # pontos de amostragem
num_signals = 10 # número de sinais
noise_level = 0.03 # nível de ruído

sin_cos_signal = signal_matrix(x -> sin(x) + cos(3*x), sample_points, noise_level, num_signals)
sin_cos_signal_raw = signal_matrix(x -> sin(x) + cos(3*x), sample_points, 0, num_signals)

plot(sample_points, sin_cos_signal, label="Sinais", xlabel="Tempo", ylabel="Amplitude", title="Sinais sin(x) + cos(3x) com ruído")

### Onda quadrada (adaptada)

$$
    x_{quadrado} =

    \begin{cases}
        1, \text{se x é par} \\
        0, \text{c.c} \\
    \end{cases}
$$

In [ ]:
# Sinal Quadrado

sample_points = 0:0.1:10 # pontos de amostragem
num_signals = 10 # número de sinais
noise_level = 0.03 # nível de ruído

square_func = x -> x % 2 < 1 ? 1 : -1
square_signal = signal_matrix(square_func, sample_points, noise_level, num_signals)
square_signal_raw = signal_matrix(square_func, sample_points, 0, num_signals)

plot(sample_points, square_signal, label="Sinais", xlabel="Tempo", ylabel="Amplitude", title="Sinais quadrado com ruído")

### Onda triangular (adaptada)

$$
    x_{triangular} = 2(x \bmod 2) - 1
$$

In [ ]:
# Sinal Triangular

sample_points = 0:0.1:10 # pontos de amostragem
num_signals = 10 # número de sinais
noise_level = 0.03 # nível de ruído

triangle_func = x -> 1 * (x % 2) - 1
triangle_signal = signal_matrix(triangle_func, sample_points, noise_level, num_signals)
triangle_signal_raw = signal_matrix(triangle_func, sample_points, 0, num_signals)

plot(sample_points, triangle_signal, label="Sinais", xlabel="Tempo", ylabel="Amplitude", title="Sinais triangular com ruído")

## SVD

### Redução de posto

In [ ]:
# Aplicando SVD para decompor os sinais em componentes singulares e reconstruir os sinais a partir de um número menor de componentes

function svd_reconstruction(signal_matrix, num_components)
    U, Σ, V = svd(signal_matrix)
    return U[:, 1:num_components] * Diagonal(Σ[1:num_components]) * V[:, 1:num_components]'
end

### $\sin(x)$

In [ ]:
plot(sample_points, sin_signal_raw[:, 1], label="", xlabel="Tempo", ylabel="Amplitude", title="sin(x)")

##### Valores singulares

In [ ]:
# Plot dos valores singulares

U, Σ, V = svd(sin_signal)

plot(Σ, label="Valores singulares", xlabel="Número do valor singular", ylabel="Valor singular", title="Valores singulares de sin(x)", seriestype = :scatter)

In [ ]:
plot(cumsum(Σ) / sum(Σ), label="", xlabel="Posto", ylabel="", title="Soma valores singulares de sin(x)", seriestype = :scatter)

#### Posto reduzido

##### Posto 1

In [ ]:
# Reconstrução do sinal senoidal com 1 componente

reconstructed_signal = svd_reconstruction(sin_signal, 1)

plot(sample_points, reconstructed_signal, label="Sinais reconstruídos")

#### Primeiro componente principal

In [ ]:
# Analisando o primeiro componente
plot(sample_points, U[:, 1], label="Primeiro componente", xlabel="Tempo", ylabel="Amplitude", title="Primeiro componente de sin(x)")

#### Diferença com o sinal original

In [ ]:
plot(sample_points, sin_signal_raw - reconstructed_signal, label="", xlabel="Tempo", ylabel="Amplitude")

#### Erro

##### Gráfico

In [ ]:
plot(sample_points, sin_signal - reconstructed_signal, label="", xlabel="Tempo", ylabel="Amplitude")

##### Valor numérico

In [ ]:
# Erro
erro_sin = norm(sin_signal - reconstructed_signal)
erro_sin_1_componente = norm(U[:, 1] - sin_signal[:, 1])

println("Erro total: ", erro_sin)
println("Erro 1 componente: ", erro_sin_1_componente)

### $\sin(x) + \cos(3x)$, $\sin(x)$ e $\cos(x/2)$

In [ ]:
sum_sin_cos_signal_raw = hcat([sin_signal_raw, sin_cos_signal_raw, cos_signal_raw]...)

plot(sample_points, sum_sin_cos_signal_raw, label="", xlabel="Tempo", ylabel="Amplitude", title="sin(x) + cos(3x), sin(x) e cos(x/2)")

In [ ]:
# Analisando caso com varios sinais trigronométricos
sum_sin_cos_signal = hcat([sin_signal, sin_cos_signal, cos_signal]...)

# Plot dos valores singulares
U, Σ, V = svd(sum_sin_cos_signal)
plot(Σ, label="Valores singulares", xlabel="Número do valor singular", ylabel="Valor singular", title="Valores singulares do sinal senoidal + cos", seriestype = :scatter)

In [ ]:
plot(cumsum(Σ) / sum(Σ), label="", xlabel="Posto", ylabel="", title="Soma valores singulares do sinal senoidal + cos", seriestype = :scatter)

In [ ]:
plot(sample_points, sum_sin_cos_signal, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais senoidal + cos")

#### Posto 1

In [ ]:
# Reconstrução do sinal sin + cos com 1 componente

reconstructed_signal_1 = svd_reconstruction(sum_sin_cos_signal, 1)
plot(sample_points, reconstructed_signal_1, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais sin + cos reconstruídos")

In [ ]:
plot(sample_points, sum_sin_cos_signal_raw - reconstructed_signal_1, label="", title="Diferença com sinal original", xlabel="Tempo", ylabel="Amplitude")

##### Erro

In [ ]:
plot(sample_points, sum_sin_cos_signal - reconstructed_signal_1, label="", xlabel="Tempo", ylabel="Amplitude")

#### Posto 2

In [ ]:
# Reconstrução do sinal senoidal + cos com 2 componente

reconstructed_signal_2 = svd_reconstruction(sum_sin_cos_signal, 2)
plot(sample_points, reconstructed_signal_2, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais senoidal + cos reconstruídos")

In [ ]:
plot(sample_points, sum_sin_cos_signal_raw - reconstructed_signal_2, label="", title="Diferença com sinal original", xlabel="Tempo", ylabel="Amplitude")

##### Erro

In [ ]:
plot(sample_points, sum_sin_cos_signal - reconstructed_signal_2, label="", xlabel="Tempo", ylabel="Amplitude")

#### Posto 3

In [ ]:
# Reconstrução do sinal senoidal + cos com 3 componente

reconstructed_signal_3 = svd_reconstruction(sum_sin_cos_signal, 3)
plot(sample_points, reconstructed_signal_3, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais senoidal + cos reconstruídos")

In [ ]:
plot(sample_points, sum_sin_cos_signal_raw - reconstructed_signal_3, label="", title="Diferença com sinal original", xlabel="Tempo", ylabel="Amplitude")

##### Erro

In [ ]:
plot(sample_points, sum_sin_cos_signal - reconstructed_signal_3, label="", xlabel="Tempo", ylabel="Amplitude")

#### Primeiro componente

In [ ]:
# Analisando o primeiro componente
plot(sample_points, U[:, 1], label="Primeiro componente", xlabel="Tempo", ylabel="Amplitude", title="Primeiro componente do sinal senoidal + cos")

#### Segundo componente

In [ ]:
# Analisando o segundo componente
plot(sample_points, U[:, 2], label="Segundo componente", xlabel="Tempo", ylabel="Amplitude", title="Segundo componente do sinal senoidal + cos")

#### Terceiro componente

In [ ]:
# Analisando o segundo componente
plot(sample_points, U[:, 3], label="Segundo componente", xlabel="Tempo", ylabel="Amplitude", title="Segundo componente do sinal senoidal + cos")

#### Matriz V

In [ ]:
display(V[1:2, :])

#### Erro em valor numérico

In [ ]:
erro_sin_cos_1 = norm(sum_sin_cos_signal - reconstructed_signal_1)
erro_sin_cos_2 = norm(sum_sin_cos_signal - reconstructed_signal_2)
erro_sin_cos_3 = norm(sum_sin_cos_signal - reconstructed_signal_3)

println("Erro total com 1 componente: ", erro_sin_cos_1)
println("Erro total com 2 componentes: ", erro_sin_cos_2)
println("Erro total com 2 componentes: ", erro_sin_cos_3)

#### Redução de dimensão

##### Dimensão 2

In [ ]:
plot(V[1,1:10], V[2,1:10], label="sin(x)", xlabel="C1", ylabel="C2", title="Análise V em dim 2", seriestype = :scatter, color="red") # sin
plot!(V[1,11:20], V[2,11:20], label="sin(x) + cos(3x)", seriestype = :scatter, color="blue") # sin + cos
plot!(V[1,21:30], V[2,21:30], label="cos(3x)", seriestype = :scatter, color="green") # cos

##### Dimensão 3

In [ ]:
plot(V[1,1:10], V[2,1:10], V[3,1:10], label="sin(x)", xlabel="C1", ylabel="C2", zlabel="C3", title="Análise V em dim 3", seriestype = :scatter, color="red") # sin
plot!(V[1,11:20], V[2,11:20], V[3,11:20], label="sin(x) + cos(3x)", seriestype = :scatter, color="blue") # sin + cos
plot!(V[1,21:30], V[2,21:30], V[3,21:30], label="cos(3x)", seriestype = :scatter, color="green") # cos

### Todos os sinais

In [ ]:
concatenate_signal_raw = hcat(sin_signal_raw, sin_cos_signal_raw, square_signal_raw, triangle_signal_raw, cos_signal_raw)

plot(sample_points, concatenate_signal_raw, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais concatenados")

In [ ]:
# Concatenando sinais sin, sin + cos, quadrado, triangular e cos

concatenate_signal = hcat(sin_signal, sin_cos_signal, square_signal, triangle_signal, cos_signal)
plot(sample_points, concatenate_signal, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais concatenados")

In [ ]:
# Plot dos valores singulares 
U, Σ, V = svd(concatenate_signal)
plot(Σ, label="Valores singulares", xlabel="Número do valor singular", ylabel="Valor singular", title="Valores singulares do sinal concatenado", seriestype = :scatter) 

In [ ]:
plot(cumsum(Σ) / sum(Σ), label="", xlabel="Posto", ylabel="", title="Soma valores singulares do sinal concatenado", seriestype = :scatter)

#### Posto 4

In [ ]:
# Reconstrução do sinal concatenado com 4 componentes

reconstructed_signal = svd_reconstruction(concatenate_signal, 4)
plot(sample_points, reconstructed_signal, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais concatenados - 4 componentes")

In [ ]:
plot(sample_points, concatenate_signal_raw - reconstructed_signal, label="", title="Diferença com sinal original", xlabel="Tempo", ylabel="Amplitude")

##### Erro

In [ ]:
plot(sample_points, concatenate_signal - reconstructed_signal, label="", xlabel="Tempo", ylabel="Amplitude")

#### Componentes

In [ ]:
# Analisando o primeiro componente
plot(sample_points, U[:, 1], label="Primeiro componente", xlabel="Tempo", ylabel="Amplitude", title="Primeiro componente do sinal concatenado")

In [ ]:
# Analisando o segundo componente
plot(sample_points, U[:, 2], label="Segundo componente", xlabel="Tempo", ylabel="Amplitude", title="Segundo componente do sinal concatenado")

In [ ]:
# Analisando o terceiro componente
plot(sample_points, U[:, 3], label="Terceiro componente", xlabel="Tempo", ylabel="Amplitude", title="Terceiro componente do sinal concatenado")

In [ ]:
# Analisando o quarto componente
plot(sample_points, U[:, 4], label="Quarto componente", xlabel="Tempo", ylabel="Amplitude", title="Quarto componente do sinal concatenado")

#### Matriz V

In [ ]:
display(V[1:4, :])

#### Erro

In [ ]:
# Erro
erro_concatenate = norm(concatenate_signal - reconstructed_signal)
println("Erro total: ", erro_concatenate)

#### Redução de dimensão

In [ ]:
plot(V[1,1:10], V[2,1:10], label="sin(x)", xlabel="C1", ylabel="C2", title="Análise V em dim 2", seriestype = :scatter, color="red") # sin
plot!(V[1,11:20], V[2,11:20], label="sin(x) + cos(3x)", seriestype = :scatter, color="blue") # sin + cos
plot!(V[1,21:30], V[2,21:30], label="Quadrado", seriestype = :scatter, color="white") # quadrado
plot!(V[1,31:40], V[2,31:40], label="Triangular", seriestype = :scatter, color="black") # triangular
plot!(V[1,41:50], V[2,41:50], label="cos(x/2)", seriestype = :scatter, color="green") # cos

In [ ]:
# Análise em 3 dimensões
plot(V[1,1:10], V[2,1:10], V[3,1:10], label="sin(x)", xlabel="C1", ylabel="C2", zlabel="C3", title="Análise V em dim 3", seriestype = :scatter, color="red") # sin
plot!(V[1,11:20], V[2,11:20], V[3,11:20], label="sin(x) + cos(3x)", seriestype = :scatter, color="blue") # sin + cos
plot!(V[1,21:30], V[2,21:30], V[3,21:30], label="Quadrado", seriestype = :scatter, color="white") # quadrado
plot!(V[1,31:40], V[2,31:40], V[3,31:40], label="Triangular", seriestype = :scatter, color="black") # triangular
plot!(V[1,41:50], V[2,41:50], V[3,41:50], label="cos(x/2)", seriestype = :scatter, color="green") # cos


# Aplicações

### Removendo ruido de sinais

In [ ]:
# # Criando um sinal ruidoso do inicio

# sample_points = 0:0.001:10 # pontos de amostragem
# num_signals = 100 # número de sinais
# noise_level = 1 # nível de ruído

# func = x -> sin(4x) + cos(2x)

# sin_real_noise = hcat([signal_vec(func, sample_points, noise_level) for i in 1:num_signals]...)

# plot(sample_points, sin_real_noise, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais senoidais com ruído")


In [ ]:
# # Plot dos valores singulares
# U, Σ, V = svd(sin_real_noise)

# plot(Σ, label="Valores singulares", xlabel="Número do valor singular", ylabel="Valor singular", title="Valores singulares de sin(x) com ruído", seriestype = :scatter)

In [ ]:
# # Reconstrução do sinal senoidal com 1 componente

# reconstructed_signal = svd_reconstruction(sin_real_noise, 1)

# plot(sample_points, reconstructed_signal, label="")

In [ ]:
# # Analisando o primeiro componente

# plot(sample_points, U[:, 1], label="Primeiro componente", xlabel="Tempo", ylabel="Amplitude", title="Primeiro componente de sin(x) com ruído")


### Audio 

In [ ]:
noise_level = 0.03

In [ ]:
# abre o arquivo de áudio
signal = wavread("0_01_0.wav")
# pega o tamanho do audio discretizado
n_sample = size(signal[1])[1]
# número de sinais ruidosos do mesmo áudio que queremos
n_signals = 100

# matriz para guardar um número n_signals de sinais ruidosos para aplicar o SVD 
noise_audio_matrix = zeros(n_sample, n_signals)
#noise_audio_matrix[:, 1] = signal[1]

for i in 1:n_signals
    noise_audio_matrix[:, i] = signal[1] .+ noise_level * rand(Normal(0, 1))
end

In [ ]:
plot(1:n_sample, signal[1], label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais de som")

In [ ]:
U, Σ, V = svd(noise_audio_matrix)
plot(Σ, label="Valores singulares", xlabel="Índice do valor singular", ylabel="Valor singular", title="Valores singulares", seriestype = :scatter)

In [ ]:
plot(1:n_sample, noise_audio_matrix, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais de som")

In [ ]:
plot(1:n_sample, U[:, 1], label="Primeiro componente", xlabel="Tempo", ylabel="Amplitude", title="Primeiro componente do sinal com som")

erro = norm( - reconstructed_signal)

In [ ]:
plot(1:n_sample, U[:, 2], label="Primeiro componente", xlabel="Tempo", ylabel="Amplitude", title="Segundo componente do sinal com som")

In [ ]:
reconstructed_signal = svd_reconstruction(noise_audio_matrix, 1)
plot(1:n_sample, reconstructed_signal, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais de som posto 1")

In [ ]:
reconstructed_signal = svd_reconstruction(noise_audio_matrix, 2)
plot(1:n_sample, reconstructed_signal, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais de som posto 2")

In [ ]:
# Transformando áudio em vetores para aplicar SVD

# Pasta 02
sample = 24000
audio_0_02 = zeros(sample, 49)
for i in 1:49
    audio_0_02[:, i] = wavread("audio/02/0_02_$i.wav")[1][1:sample]*randn()
end



In [ ]:
# Pasta 25

audio_0_25 = zeros(sample, 49)
for i in 1:49
    audio_0_25[:, i] = wavread("audio/25/0_25_$i.wav")[1][1:sample]
end

In [ ]:
# Pasta 28

audio_0_28 = zeros(sample, 49)
for i in 1:49
    audio_0_28[:, i] = wavread("audio/28/0_28_$i.wav")[1][1:sample]
end

In [ ]:
audio_0 = hcat(audio_0_02, audio_0_25, audio_0_28)
size(audio_0)

In [ ]:
# Plot dos valores singulares
U, Σ, V = svd(audio_0)
plot(Σ, label="Valores singulares", xlabel="Número do valor singular", ylabel="Valor singular", title="Valores singulares do áudio 02", seriestype = :scatter)

In [ ]:
# Plot dos valores singulares
U, Σ, V = svd(audio_0_02)
plot(Σ, label="Valores singulares", xlabel="Número do valor singular", ylabel="Valor singular", title="Valores singulares do áudio 25", seriestype = :scatter)

In [ ]:
# Plot dos sinais por pessoa
plot(audio_0_02, label="", xlabel="Tempo", ylabel="Amplitude", title="Sinais de áudio 02")

In [ ]:
# Adicionando ruído ao sinal de áudio
wavwrite(audio_0_noise[:, 1], "audio_0_noise.wav", Fs=24000)
